In [1]:
import numpy as np
import torch
import math
import os
import pickle
import modules.utils as utils
import modules.models as models
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
from torchvision import transforms
from pathlib import Path
import torch.nn.init as init
from torch.nn.parameter import Parameter
from torchvision.datasets import FashionMNIST


Bad key text.latex.unicode in file /Users/nirweingarten/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 112 ('text.latex.unicode : False # use "ucs" and "inputenc" LaTeX packages for handling')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.2/matplotlibrc.template
or from the matplotlib source distribution

Bad key savefig.frameon in file /Users/nirweingarten/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 423 ('savefig.frameon : True')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.2/matplotlibrc.template
or from the matplotlib source distribution

Bad key pgf.debug in file /Users/nirweingarten/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 444 ('pgf.de

no umap installed, proceeding...


In [4]:
DATA_DIR = os.path.join('..', 'datasets', 'FMNIST_DATASET')
BATCH_SIZE = 100
SEED = 42
LR = 1e-4 # learning rate
BETA = 1e-3 # rate distortion hyperparam
BETA_DECAY = 0.999 # EMA exponential decay
SAVE_DIR = '..'
LOAD_MODEL = False
MC_SAMPLE_SIZE = 12
EPOCHS = 200 # in the paprt they did 200 epochs

In [3]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

if os.path.isdir(DATA_DIR):
    try:
        train_data = FashionMNIST(root=DATA_DIR, train=True, transform=transforms.ToTensor())
        test_data = FashionMNIST(root=DATA_DIR, train=False, transform=transforms.ToTensor())
    except:
        train_data = FashionMNIST(root=DATA_DIR, train=True, download=True, transform=transforms.ToTensor())
        test_data = FashionMNIST(root=DATA_DIR, train=False, download=True, transform=transforms.ToTensor())
else:
    os.mkdir(DATA_DIR)
    train_data = FashionMNIST(root=DATA_DIR, train=True, download=True, transform=transforms.ToTensor())
    test_data = FashionMNIST(root=DATA_DIR, train=False, download=True, transform=transforms.ToTensor())

train_loader = DataLoader(train_data,
                          batch_size=BATCH_SIZE,
                          shuffle=True,
                          num_workers=1,
                          drop_last=True)

test_loader = DataLoader(test_data,
                         batch_size=BATCH_SIZE,
                         shuffle=False,
                         num_workers=1,
                         drop_last=False)

fmnist_classes = ('T-shirt/top', 'Trouser', 'Pullover', 'Dress',
           'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot')

Using downloaded and verified file: ../datasets/FMNIST_DATASET/FashionMNIST/raw/train-images-idx3-ubyte.gz
Extracting ../datasets/FMNIST_DATASET/FashionMNIST/raw/train-images-idx3-ubyte.gz to ../datasets/FMNIST_DATASET/FashionMNIST/raw
Using downloaded and verified file: ../datasets/FMNIST_DATASET/FashionMNIST/raw/train-labels-idx1-ubyte.gz
Extracting ../datasets/FMNIST_DATASET/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ../datasets/FMNIST_DATASET/FashionMNIST/raw
Using downloaded and verified file: ../datasets/FMNIST_DATASET/FashionMNIST/raw/t10k-images-idx3-ubyte.gz
Extracting ../datasets/FMNIST_DATASET/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ../datasets/FMNIST_DATASET/FashionMNIST/raw
Using downloaded and verified file: ../datasets/FMNIST_DATASET/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz
Extracting ../datasets/FMNIST_DATASET/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ../datasets/FMNIST_DATASET/FashionMNIST/raw
Processing...


/Users/nirweingarten/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Done!


### Vanilla IB VAE: k=256

In [6]:
K = 256
MODEL_SAVE_PATH = os.path.join('..', 'saved_models', 'fmnist_vanila_k256.h5')
EMA_SAVE_PATH = os.path.join('..', 'saved_models', 'fmnist_vanila_k256_ema.h5')
PICKLE_SAVE_PATH = os.path.join('..', 'pickles', 'fmnist_vanila_k256.pkl')
torch.manual_seed(SEED)
np.random.seed(SEED)
model = models.MNIST_VANILA_IB_VAE(K, device).to(device)
model_ema = utils.EMA_smoothning(models.MNIST_VANILA_IB_VAE(K, device).to(device), model.state_dict(), beta_decay=BETA_DECAY)
book_keeper = utils.train_model(EPOCHS, model, model_ema, train_loader, test_loader, device,
                  LR, MODEL_SAVE_PATH, EMA_SAVE_PATH, PICKLE_SAVE_PATH, BETA)

[epoch 0]
i:1 IZY:-0.82 IZX:1752.37 acc:0.1300 err:0.8700
i:101 IZY:2.61 IZX:571.90 acc:0.8700 err:0.1300
i:201 IZY:2.49 IZX:425.90 acc:0.8000 err:0.2000
i:301 IZY:2.52 IZX:402.03 acc:0.8100 err:0.1900
i:401 IZY:2.63 IZX:336.48 acc:0.8400 err:0.1600
i:501 IZY:2.56 IZX:288.86 acc:0.7800 err:0.2200
[TEST RESULTS]
IZY:2.65 IZX:305.10 acc:0.8355 err:0.1645
[epoch 1]
i:1 IZY:2.75 IZX:312.33 acc:0.8700 err:0.1300
i:101 IZY:2.44 IZX:264.92 acc:0.8200 err:0.1800
i:201 IZY:2.75 IZX:278.74 acc:0.8700 err:0.1300
i:301 IZY:2.80 IZX:230.06 acc:0.8900 err:0.1100
i:401 IZY:2.67 IZX:252.88 acc:0.8000 err:0.2000
i:501 IZY:2.52 IZX:292.96 acc:0.8100 err:0.1900
[TEST RESULTS]
IZY:2.65 IZX:216.88 acc:0.8387 err:0.1613
[epoch 2]
i:1 IZY:2.61 IZX:215.46 acc:0.8300 err:0.1700
i:101 IZY:2.65 IZX:213.73 acc:0.8600 err:0.1400
i:201 IZY:2.76 IZX:224.14 acc:0.8700 err:0.1300
i:301 IZY:3.04 IZX:204.84 acc:0.9300 err:0.0700
i:401 IZY:2.90 IZX:201.71 acc:0.8900 err:0.1100
i:501 IZY:2.78 IZX:222.02 acc:0.8700 err:0.1

### GMM IB VAE: k=128, w=2

In [7]:
K = 128
W = 2
MODEL_SAVE_PATH = os.path.join('..', 'saved_models', 'fmnist_gmm_k128_w2.h5')
EMA_SAVE_PATH = os.path.join('..', 'saved_models', 'fmnist_gmm_ema_k128_w2.h5')
PICKLE_SAVE_PATH = os.path.join('..', 'pickles', 'fmnist_stats_gmm_k128_w2.pkl')
torch.manual_seed(SEED)
np.random.seed(SEED)
model = models.MNIST_IB_VAE_GMM(K, W, device).to(device)
model_ema = utils.EMA_smoothning(models.MNIST_IB_VAE_GMM(K, W, device).to(device), model.state_dict(), beta_decay=BETA_DECAY)
book_keeper = utils.train_model(EPOCHS, model, model_ema, train_loader, test_loader, device,
                  LR, MODEL_SAVE_PATH, EMA_SAVE_PATH, PICKLE_SAVE_PATH, BETA)

[epoch 0]
i:1 IZY:-0.82 IZX:877.38 acc:0.1100 err:0.8900
i:101 IZY:2.38 IZX:325.05 acc:0.8100 err:0.1900
i:201 IZY:2.41 IZX:256.17 acc:0.7500 err:0.2500
i:301 IZY:2.65 IZX:228.74 acc:0.8500 err:0.1500
i:401 IZY:2.71 IZX:190.92 acc:0.8600 err:0.1400
i:501 IZY:2.53 IZX:178.54 acc:0.8300 err:0.1700
[TEST RESULTS]
IZY:2.67 IZX:173.98 acc:0.8353 err:0.1647
[epoch 1]
i:1 IZY:2.77 IZX:160.97 acc:0.8500 err:0.1500
i:101 IZY:2.87 IZX:166.29 acc:0.8600 err:0.1400
i:201 IZY:2.81 IZX:144.98 acc:0.8800 err:0.1200
i:301 IZY:2.62 IZX:162.90 acc:0.8700 err:0.1300
i:401 IZY:2.64 IZX:141.76 acc:0.8000 err:0.2000
i:501 IZY:2.63 IZX:127.53 acc:0.8500 err:0.1500
[TEST RESULTS]
IZY:2.75 IZX:150.85 acc:0.8512 err:0.1488
[epoch 2]
i:1 IZY:2.76 IZX:151.35 acc:0.8600 err:0.1400
i:101 IZY:2.93 IZX:136.13 acc:0.9100 err:0.0900
i:201 IZY:2.68 IZX:146.09 acc:0.8100 err:0.1900
i:301 IZY:2.84 IZX:129.71 acc:0.9000 err:0.1000
i:401 IZY:2.98 IZX:137.04 acc:0.9300 err:0.0700
i:501 IZY:2.52 IZX:132.18 acc:0.8400 err:0.16

In [8]:
torch.softmax(model.gmm.weights, dim=1)

tensor([[0.5008, 0.4992]], device='cuda:0', grad_fn=<SoftmaxBackward>)

### GMM IB VAE: k=64, w=4

In [9]:
K = 64
W = 4
MODEL_SAVE_PATH = os.path.join('..', 'saved_models', 'fmnist_gmm_k64_w4.h5')
EMA_SAVE_PATH = os.path.join('..', 'saved_models', 'fmnist_gmm_ema_k64_w4.h5')
PICKLE_SAVE_PATH = os.path.join('..', 'pickles', 'fmnist_stats_gmm_k64_w4.pkl')
torch.manual_seed(SEED)
np.random.seed(SEED)
model = models.MNIST_IB_VAE_GMM(K, W, device).to(device)
model_ema = utils.EMA_smoothning(models.MNIST_IB_VAE_GMM(K, W, device).to(device), model.state_dict(), beta_decay=BETA_DECAY)
book_keeper = utils.train_model(EPOCHS, model, model_ema, train_loader, test_loader, device,
                  LR, MODEL_SAVE_PATH, EMA_SAVE_PATH, PICKLE_SAVE_PATH, BETA)

[epoch 0]
i:1 IZY:-0.25 IZX:446.81 acc:0.0500 err:0.9500
i:101 IZY:2.21 IZX:191.42 acc:0.7400 err:0.2600
i:201 IZY:2.60 IZX:148.81 acc:0.8200 err:0.1800
i:301 IZY:2.65 IZX:114.42 acc:0.8600 err:0.1400
i:401 IZY:2.72 IZX:110.21 acc:0.8400 err:0.1600
i:501 IZY:2.83 IZX:116.57 acc:0.8800 err:0.1200
[TEST RESULTS]
IZY:2.71 IZX:106.67 acc:0.8463 err:0.1537
[epoch 1]
i:1 IZY:2.84 IZX:110.37 acc:0.8900 err:0.1100
i:101 IZY:2.75 IZX:108.10 acc:0.8500 err:0.1500
i:201 IZY:2.79 IZX:97.12 acc:0.8700 err:0.1300
i:301 IZY:2.83 IZX:109.04 acc:0.8600 err:0.1400
i:401 IZY:2.72 IZX:94.27 acc:0.8300 err:0.1700
i:501 IZY:2.74 IZX:90.39 acc:0.8600 err:0.1400
[TEST RESULTS]
IZY:2.73 IZX:96.93 acc:0.8445 err:0.1555
[epoch 2]
i:1 IZY:2.85 IZX:97.63 acc:0.8900 err:0.1100
i:101 IZY:2.81 IZX:97.17 acc:0.8700 err:0.1300
i:201 IZY:2.88 IZX:91.64 acc:0.9100 err:0.0900
i:301 IZY:2.86 IZX:81.19 acc:0.9100 err:0.0900
i:401 IZY:3.00 IZX:87.17 acc:0.9300 err:0.0700
i:501 IZY:2.80 IZX:82.26 acc:0.8400 err:0.1600
[TEST R

In [10]:
torch.softmax(model.gmm.weights, dim=1)

tensor([[0.2300, 0.2356, 0.2790, 0.2554]], device='cuda:0',
       grad_fn=<SoftmaxBackward>)

### 2 sequential stochastic layers, k=256



In [11]:
K = 256
MODEL_SAVE_PATH = os.path.join('..', 'saved_models', 'fmnist_seq_k256.h5')
EMA_SAVE_PATH = os.path.join('..', 'saved_models', 'fmnist_seq_ema_k256.h5')
PICKLE_SAVE_PATH = os.path.join('..', 'pickles', 'fmnist_stats_seq_k256.pkl')
torch.manual_seed(SEED)
np.random.seed(SEED)
model = models.MNIST_2STOCHASTIC(K, device).to(device)
model_ema = utils.EMA_smoothning(models.MNIST_2STOCHASTIC(K, device).to(device), model.state_dict(), beta_decay=BETA_DECAY)
book_keeper = utils.train_model(EPOCHS, model, model_ema, train_loader, test_loader, device,
                  LR, MODEL_SAVE_PATH, EMA_SAVE_PATH, PICKLE_SAVE_PATH, BETA)

[epoch 0]
i:1 IZY:-0.76 IZX:2626.27 acc:0.1500 err:0.8500
i:101 IZY:2.31 IZX:1319.16 acc:0.7900 err:0.2100
i:201 IZY:2.43 IZX:1216.39 acc:0.7500 err:0.2500
i:301 IZY:2.71 IZX:1159.23 acc:0.8400 err:0.1600
i:401 IZY:2.70 IZX:1092.07 acc:0.8500 err:0.1500
i:501 IZY:2.47 IZX:1086.08 acc:0.7800 err:0.2200
[TEST RESULTS]
IZY:2.66 IZX:1063.62 acc:0.8316 err:0.1684
[epoch 1]
i:1 IZY:2.71 IZX:1060.23 acc:0.8500 err:0.1500
i:101 IZY:2.88 IZX:1053.64 acc:0.8900 err:0.1100
i:201 IZY:2.79 IZX:1035.25 acc:0.8500 err:0.1500
i:301 IZY:2.64 IZX:1014.50 acc:0.8500 err:0.1500
i:401 IZY:2.86 IZX:1015.96 acc:0.8900 err:0.1100
i:501 IZY:2.94 IZX:1002.46 acc:0.8800 err:0.1200
[TEST RESULTS]
IZY:2.73 IZX:1026.75 acc:0.8463 err:0.1537
[epoch 2]
i:1 IZY:2.71 IZX:1037.99 acc:0.8200 err:0.1800
i:101 IZY:2.76 IZX:1000.93 acc:0.8400 err:0.1600
i:201 IZY:3.03 IZX:1011.83 acc:0.9200 err:0.0800
i:301 IZY:2.85 IZX:988.51 acc:0.9200 err:0.0800
i:401 IZY:2.87 IZX:983.89 acc:0.8800 err:0.1200
i:501 IZY:2.67 IZX:1002.19 a

### 2 sequential stochastic layers + intermediate linear layer, k=256

In [12]:
K = 256
MODEL_SAVE_PATH = os.path.join('..', 'saved_models', 'fmnist_seq_extened_k256.h5')
EMA_SAVE_PATH = os.path.join('..', 'saved_models', 'fmnist_seq_extened_ema_k256.h5')
PICKLE_SAVE_PATH = os.path.join('..', 'pickles', 'fmnist_stats_seq_extened_k256.pkl')
torch.manual_seed(SEED)
np.random.seed(SEED)
model = models.MNIST_2STOCHASTIC_EXTENDED(K, device).to(device)
model_ema = utils.EMA_smoothning(models.MNIST_2STOCHASTIC_EXTENDED(K, device).to(device), model.state_dict(), beta_decay=BETA_DECAY)
book_keeper = utils.train_model(EPOCHS, model, model_ema, train_loader, test_loader, device,
                  LR, MODEL_SAVE_PATH, EMA_SAVE_PATH, PICKLE_SAVE_PATH, BETA)

[epoch 0]
i:1 IZY:-1.11 IZX:3464.08 acc:0.1100 err:0.8900
i:101 IZY:2.25 IZX:1697.09 acc:0.7600 err:0.2400
i:201 IZY:2.30 IZX:1234.22 acc:0.8100 err:0.1900
i:301 IZY:2.20 IZX:1095.51 acc:0.7600 err:0.2400
i:401 IZY:2.21 IZX:1004.67 acc:0.7800 err:0.2200
i:501 IZY:2.62 IZX:989.03 acc:0.8400 err:0.1600
[TEST RESULTS]
IZY:2.65 IZX:926.14 acc:0.8359 err:0.1641
[epoch 1]
i:1 IZY:2.61 IZX:889.24 acc:0.8300 err:0.1700
i:101 IZY:2.71 IZX:865.37 acc:0.8600 err:0.1400
i:201 IZY:2.69 IZX:793.07 acc:0.8700 err:0.1300
i:301 IZY:2.63 IZX:746.35 acc:0.8200 err:0.1800
i:401 IZY:2.72 IZX:791.99 acc:0.8700 err:0.1300
i:501 IZY:2.71 IZX:732.85 acc:0.8700 err:0.1300
[TEST RESULTS]
IZY:2.70 IZX:773.31 acc:0.8460 err:0.1540
[epoch 2]
i:1 IZY:2.66 IZX:757.94 acc:0.8600 err:0.1400
i:101 IZY:2.67 IZX:776.13 acc:0.8400 err:0.1600
i:201 IZY:2.63 IZX:749.11 acc:0.8200 err:0.1800
i:301 IZY:2.94 IZX:666.17 acc:0.9000 err:0.1000
i:401 IZY:2.59 IZX:697.19 acc:0.8400 err:0.1600
i:501 IZY:2.69 IZX:649.59 acc:0.8800 err